In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data_train = pd.read_csv(r'C:\Users\ckatkaa\Desktop\jupyter notebook\Hack 2\data\train.csv')
data_test = pd.read_csv(r'C:\Users\ckatkaa\Desktop\jupyter notebook\Hack 2\data\test.csv')

In [3]:
data_train.shape,data_test.shape

((7274, 3), (1819, 2))

In [4]:
data_train.head()

,tweet_id,tweet,sentiment
0,1701,#sxswnui #sxsw #apple defining language of tou...,1
1,1851,Learning ab Google doodles! All doodles should...,1
2,2689,one of the most in-your-face ex. of stealing t...,2
3,4525,This iPhone #SXSW app would b pretty awesome i...,0
4,3604,Line outside the Apple store in Austin waiting...,1


In [5]:
data_train.isnull().sum()

tweet_id     0
tweet        1
sentiment    0
dtype: int64

In [6]:
data_train.drop(['tweet_id'],axis=1,inplace=True)
data_test.drop(['tweet_id'],axis=1,inplace=True)

In [7]:
data_train.isnull().sum()

tweet        1
sentiment    0
dtype: int64

In [8]:
data_test.isnull().sum()

tweet    0
dtype: int64

In [9]:
data_train['tweet']=np.where(pd.isnull(data_train['tweet']),'None',data_train['tweet'])

In [10]:
data_train.groupby(['sentiment']).count()

,tweet
sentiment,
0,456
1,4311
2,2382
3,125


In [11]:
import nltk

In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ckatkaa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords
stop= set(stopwords.words('english')) 

data_train['processedtext'] = data_train['tweet'].str.replace('[^\w\s]','')
data_train['processedtext'] = data_train['processedtext'].str.replace('@\w+','')
data_train['processedtext'] = data_train['processedtext'].str.replace('#\w+','_mention_')
data_train['processedtext'] = data_train['processedtext'].str.replace('\d\d\/\d\d\/\d\d\d\d','__date__')
data_train['processedtext'] = data_train['processedtext'].astype(str).apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data_train['processedtext'] = data_train['processedtext'].astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))

data_test['processedtext'] = data_test['tweet'].str.replace('[^\w\s]','')
data_test['processedtext'] = data_test['processedtext'].str.replace('@\w+','')
data_test['processedtext'] = data_test['processedtext'].str.replace('#\w+','_mention_')
data_test['processedtext'] = data_test['processedtext'].str.replace('\d\d\/\d\d\/\d\d\d\d','__date__')
data_test['processedtext'] = data_test['processedtext'].astype(str).apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data_test['processedtext'] = data_test['processedtext'].astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))


In [14]:
data_train.head()

,tweet,sentiment,processedtext
0,#sxswnui #sxsw #apple defining language of tou...,1,sxswnui sxsw apple defining language touch dif...
1,Learning ab Google doodles! All doodles should...,1,learning ab google doodles all doodles light f...
2,one of the most in-your-face ex. of stealing t...,2,one inyourface ex stealing show yrs rt mention...
3,This iPhone #SXSW app would b pretty awesome i...,0,this iphone sxsw app would b pretty awesome di...
4,Line outside the Apple store in Austin waiting...,1,line outside apple store austin waiting new ip...


In [15]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ckatkaa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
data_train['processedtext'] = data_train['processedtext'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))
data_test['processedtext'] = data_test['processedtext'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))

In [17]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
data_train['processedtext'] = data_train['processedtext'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))
data_test['processedtext'] = data_test['processedtext'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))

In [18]:
data_train.head()

,tweet,sentiment,processedtext
0,#sxswnui #sxsw #apple defining language of tou...,1,sxswnui sxsw appl defin languag touch differ d...
1,Learning ab Google doodles! All doodles should...,1,learn ab googl doodl all doodl light funni amp...
2,one of the most in-your-face ex. of stealing t...,2,one inyourfac ex steal show yr rt mention quot...
3,This iPhone #SXSW app would b pretty awesome i...,0,thi iphon sxsw app would b pretti awesom didnt...
4,Line outside the Apple store in Austin waiting...,1,line outsid appl store austin wait new ipad sx...


In [19]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', ngram_range=(1, 2),
                        lowercase=True, stop_words= 'english')

data_tfidf_train = tfidf.fit_transform(data_train['processedtext'])

In [20]:
data_tfidf_train=data_tfidf_train.toarray()

In [21]:
data_tfidf_train.shape

(7274, 3615)

In [22]:
df1 = pd.DataFrame(data_tfidf_train, columns=tfidf.get_feature_names())

In [23]:
df1.columns

Index(['031111', '031111 infektd', '031211', '031211 infektd', '031311',
       '031311 infektd', '031411', '031411 infektd', '10', '10 austin',
       ...
       'ûïmention', 'ûïmention appl', 'ûïmention googl', 'ûïmention mention',
       'ûïmention new', 'ûïmention rt', 'ûïmention rumor', 'ûïmention sxsw',
       'ûò', 'ûó'],
      dtype='object', length=3615)

In [24]:
data_tfidf_test = tfidf.transform(data_test['processedtext'])
df2 = pd.DataFrame(data_tfidf_test.toarray(), columns=tfidf.get_feature_names())

In [25]:
data_train.drop(['tweet','processedtext'],axis=1,inplace=True)
data_test.drop(['tweet','processedtext'],axis=1,inplace=True)

In [26]:
data_train1=pd.concat([data_train,df1],axis=1)
data_test1=pd.concat([data_test,df2],axis=1)

In [27]:
data_train1.shape,data_test1.shape

((7274, 3616), (1819, 3615))

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [29]:
X=data_train1.drop(['sentiment'],axis=1)
y=data_train1['sentiment']

In [30]:
X

,031111,031111 infektd,031211,031211 infektd,031311,031311 infektd,031411,031411 infektd,10,10 austin,...,ûïmention,ûïmention appl,ûïmention googl,ûïmention mention,ûïmention new,ûïmention rt,ûïmention rumor,ûïmention sxsw,ûò,ûó
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state=0)

In [32]:
y_train.value_counts()

1    3004
2    1671
0     324
3      92
Name: sentiment, dtype: int64

In [33]:
y_test.value_counts()

1    1307
2     711
0     132
3      33
Name: sentiment, dtype: int64

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [35]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
predict1=lr.predict(X_test)

In [36]:
print(classification_report(predict1,y_test))

              precision    recall  f1-score   support

           0       0.05      0.54      0.10        13
           1       0.87      0.69      0.77      1662
           2       0.44      0.61      0.51       508
           3       0.00      0.00      0.00         0

    accuracy                           0.67      2183
   macro avg       0.34      0.46      0.34      2183
weighted avg       0.77      0.67      0.71      2183



C:\Users\ckatkaa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
predict2=lr.predict(data_test1)

In [38]:
data_test = pd.read_csv(r'C:\Users\ckatkaa\Desktop\jupyter notebook\Hack 2\data\test.csv')

In [39]:
data_test['sentiment']=pd.DataFrame(predict2)

In [40]:
data_test = data_test.drop('tweet',1)

In [41]:
data_test['sentiment'].value_counts()

1    1353
2     457
0       9
Name: sentiment, dtype: int64

In [42]:
data_test.to_csv(r'C:\Users\ckatkaa\Desktop\jupyter notebook\Hack 2\data\Output.csv',index=False)

# Treating on imbalance Data

In [43]:
#conda install -c conda-forge imbalanced-learn

In [44]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)

In [45]:
X_train1,X_test1,y_train1,y_test1=train_test_split(X_resampled, y_resampled,test_size=0.3)

In [46]:
lr.fit(X_train1,y_train1)
predict_over=lr.predict(X_test1)
print(classification_report(y_test1,predict_over))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90      1291
           1       0.72      0.65      0.68      1268
           2       0.75      0.69      0.72      1284
           3       0.92      1.00      0.96      1331

    accuracy                           0.82      5174
   macro avg       0.81      0.82      0.81      5174
weighted avg       0.81      0.82      0.82      5174



C:\Users\ckatkaa\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Classification Report on Train Data

In [47]:
predict_over2=lr.predict(X_train1)
print(classification_report(y_train1,predict_over2))

              precision    recall  f1-score   support

           0       0.90      0.95      0.93      3020
           1       0.84      0.78      0.80      3043
           2       0.84      0.81      0.83      3027
           3       0.94      1.00      0.97      2980

    accuracy                           0.88     12070
   macro avg       0.88      0.88      0.88     12070
weighted avg       0.88      0.88      0.88     12070



Fit Same Model on imbalance dataset(old)

In [48]:
print(classification_report(y_test,lr.predict(X_test)))


              precision    recall  f1-score   support

           0       0.50      0.95      0.65       132
           1       0.89      0.74      0.81      1307
           2       0.71      0.76      0.73       711
           3       0.41      1.00      0.58        33

    accuracy                           0.76      2183
   macro avg       0.63      0.86      0.69      2183
weighted avg       0.80      0.76      0.77      2183



In [49]:
final_predict=lr.predict(data_test1)

In [50]:
data_test['sentiment']=pd.DataFrame(final_predict)

In [51]:
data_test

,tweet_id,sentiment
0,7506,2
1,7992,1
2,247,1
3,7688,2
4,3294,2
5,6125,1
6,6131,2
7,4134,1
8,8206,0
9,8552,1


In [52]:
data_test['sentiment'].value_counts()

1    938
2    619
0    208
3     54
Name: sentiment, dtype: int64

In [53]:
data_test.to_csv(r'C:\Users\ckatkaa\Desktop\jupyter notebook\Hack 2\Final Submission\Try_2\Output.csv',index=False)

SMOTE OverSampling

In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=777, k_neighbors=1)
X_SMOTE, y_SMOTE = smt.fit_sample(X,y)
# pd.DataFrame(X_SMOTE.todense(), columns=tv.get_feature_names())

In [ ]:
X_train2,X_test2,y_train2,y_test2=train_test_split(X_SMOTE, y_SMOTE,test_size=0.3)

In [ ]:
lr.fit(X_train2,y_train2)
predict_over_smote=lr.predict(X_test2)
print(classification_report(y_test2,predict_over_smote))

In [ ]:
predict_over_smote2=lr.predict(X_train2)
print(classification_report(y_train2,predict_over_smote2))

In [ ]:
final_predict_by_smote=lr.predict(data_test1)

In [ ]:
data_test['sentiment']=pd.DataFrame(final_predict_by_smote)